## Imports and chains

In [1]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from cot import Collection
import json
from langchain.chat_models import ChatOpenAI

In [107]:
llm = OpenAI(temperature=.0,model_name="text-davinci-003")   #gpt-3.5-turbo #Chat

reflect_template = """
    Question: {question}
    Answer_choices: {answer_choices}

    Reasoning: {cot_trigger}{cot}
    {answer_extraction}
    Answer: {answer}
    
    {reflection_prompt}
    """
prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt'], template=reflect_template)
reflect_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection")

#{instruction}
extraction_template = """

    Question: {question}
    Answer_choices: {answer_choices}

    Reasoning: {cot_trigger}{cot}
    {answer_extraction}{answer}
    
    Reflection: {reflection_prompt}{reflection}

    {reflect_answer_extraction}
    """
    #Get reflection
prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt','reflection','reflect_answer_extraction'], template=extraction_template)
reflect_answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection_answer")

    # This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
reflect_overall_chain = SequentialChain(chains=[reflect_chain, reflect_answer_chain],input_variables=["question","answer_choices","cot_trigger","answer_extraction",'cot','answer','reflection_prompt','reflect_answer_extraction'],
        output_variables=["reflection", "reflection_answer"],
        verbose=True)



## Data


In [55]:
cot_dataset = Collection.from_json("worldtree_10.json") #input dataset
worldtree_10 = cot_dataset.select(split="train", number_samples = 10, random_samples=True, seed=0)

In [95]:
coll = Collection.load_thoughtsource_100(names='worldtree',load_pregenerated_cots=True)
coll = coll.select(split="all", number_samples=30)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='openai') #have one

In [91]:
#coll['worldtree']['test'][0]

In [97]:
coll['worldtree']['test'][0]['generated_cot']

[{'id': 'fec9713b-7855-4e87-b9fc-3245cbe493cf',
  'fragments_version': '0.01',
  'instruction': None,
  'cot_trigger': 'kojima-01',
  'cot_trigger_template': '{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}',
  'prompt_text': '',
  'cot': ' First, gravity causes gases and dust particles to condense into spheres. Then, as the particles become more and more dense, gravity pulls them together and converts solid matter into gases and light energy. Therefore, the best answer is B) Gravity causes gases and dust particles to condense into spheres.',
  'answers': [{'id': '9978c9b9-e528-4d9e-8065-c5c131eec95a',
    'answer_extraction': 'kojima-A-D',
    'answer_extraction_template': '{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}{cot}\n{answer_extraction}',
    'answer_extraction_text': '',
    'answer': ' B.',
    'correct_answer': True}],
  'author': 'thoughtsource',
  'date': '2023/02/10 18:41:59',
  'api_service': 'openai',
  'model': "{'name': 'text-davinci-0

## Run reflection

In [ ]:
input_dict = {
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is", 
    'answer':"", 
    'cot': "", 
    'reflection_prompt':"Double check this",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?'
}
metareason = coll.metareason_flexible(chain=reflect_overall_chain,input_dict=input_dict)

In [100]:
#Force langchain into TS structure 
worldtree_new = {'worldtree':{'test':metareason}}

#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_new, outfile)
collect = Collection.from_json('sample.json')

collect

| Name      | Train   | Valid   |   Test |
|-----------|---------|---------|--------|
| worldtree | -       | -       |     30 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [ ]:
#Conclusion: with these prompts, there is no new information and no changed answers

In [108]:
input_dict = {
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is", 
    'answer':"", 
    'cot': "", 
    'reflection_prompt':"Critique the reasoning above",
    'reflect_answer_extraction':'Based on the reasoning and the reflection, what is the final answer? (A-D)?'
}
metareason = coll.metareason_flexible(chain=reflect_overall_chain,input_dict=input_dict)

Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


In [109]:
#Force langchain into TS structure 
worldtree_new = {'worldtree':{'test':metareason}}

#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_new, outfile)
collect = Collection.from_json('sample.json')

collect

| Name      | Train   | Valid   |   Test |
|-----------|---------|---------|--------|
| worldtree | -       | -       |     30 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [ ]:
#chatgpt seems to be a bad corrector, what about its competitors?
#but davinci is no better

In [54]:
#example cot and ans

metareason[0]['generated_cot'][3]
metareason[0]['generated_cot'][3]['answers']

[{'id': '8b8ddb55-e1fc-4fac-85c7-333a76cacd91',
  'answer_extraction': 'Therefore, among A through D, the answer is',
  'answer_extraction_template': '',
  'answer_extraction_text': 'self_reflection',
  'answer': 'The definite answer is B) There is a change in the appearance of the objects.',
  'correct_answer': None}]

Next: evaluate in annotator